In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark stochastics and (possible) collision

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(4)
print(M)
from src.plotting import *

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma = jnp.diag(jnp.array([.5,.5]))
n_steps = 5000

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),np.zeros(M.N))).T.flatten())

(ts,xs,charts) = M.Brownian_coords(q,dWs(M.dim,n_steps=n_steps))

# plot
M.newfig()
M.plot()
M.plot_path(zip(xs,charts))
plt.show()

# Simulations in various dimensions

In [ ]:
def simBrownianDistances(m,N,Nsamples,scale=.5,mindist=.25):
    global key
    
    # dimension
    M.setm(m,jnp.diag(jnp.array([.5]*d)))
    
    # number of landmarks
    M.setN(N)
    
    # configuration
#     q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),jnp.zeros((d-1,M.N)))).T.flatten())
    qs = np.zeros((Nsamples,M.dim))
    for i in range(Nsamples):
        # sample until configuration with min landmark distances found
        found = False
        while not found:
            subkey, key = jax.random.split(key)
            q = scale*random.normal(subkey,(M.N,M.m))
            mindistq = np.inf
            for k in range(0,N):
                for l in range(k+1,N):
                    mindistq = np.min((mindistq,np.linalg.norm(q[k]-q[l])))
            if mindistq > mindist:
                found = True
        qs[i] = q.flatten()

    # generate multiple sample paths
    xss = np.zeros((Nsamples,n_steps,M.dim))
    chartss = np.zeros((Nsamples,n_steps,q[1].shape[0]))
    for i in range(Nsamples):
        (ts,xs,charts) = M.Brownian_coords(M.coords(qs[i]),dWs(M.dim,n_steps=n_steps))
        xss[i] = xs
        chartss[i] = charts

    # plot
    colormap = plt.get_cmap('winter')
    colors=[colormap(k) for k in np.linspace(0, 1, Nsamples)]
#     if d == 1:
#         M.newfig()
#         M.plot()
#         for i in range(Nsamples):
#             plt.plot(xss[i][:,0],xss[i][:,1],color=colors[i])
#         plt.plot(q[0][0],q[0][1],'o',color='r',markersize=10)
#         plt.show()
#     elif d == 2:
#         # plot
#         M.newfig()
#         M.plot()
#         for i in range(Nsamples):
#             M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
#         M.plotx(q,color='r')
#         plt.show()
        
    minv,mini = jnp.inf,-1

    # plot min distance
    d2s = np.zeros((Nsamples,n_steps))
    for i in range(Nsamples):
        xs = xss[i].reshape((n_steps,M.N,M.m))
        d2 = np.zeros((n_steps,N*(N-1)//2))
        # loop over pairs of dimensions and take minimum
        r = 0
        for k in range(0,N):
            for l in range(k+1,N):
                d2[:,r] = jnp.sum((xs[:,k]-xs[:,l])**2,1)
                r = r+1
        d2 = np.min(d2,1)
        d2s[i] = d2
        if jnp.nanmin(d2) < minv:
            minv, mini = jnp.nanmin(d2), i
        plt.plot(jnp.cumsum(dts(n_steps=n_steps)),jnp.log(d2),color=colors[i])
    plt.title("%s kernel, N %d, dimension %d, %d sample paths" % (kernel,N,d,Nsamples))
    plt.xlabel('t'), plt.ylabel('log(distance^2)')
    plt.show()
    
    print("min distance^2,log(distance^2):",jnp.nanmin(d2s[mini]),jnp.nanmin(jnp.log(d2s[mini])))
    
    # finding percentiles
    pcts = (np.array([0, 20, 35, 45, 55, 65, 80, 99])/100*Nsamples).astype('int')
    layers = np.empty([n_steps, 8])
    for i in range(n_steps) : 
        _sorted = np.sort(jnp.log(d2s[:,i]))
        layers[i, :] = _sorted[pcts]

    # plot the layers
    for i in range(7) :
        plt.fill_between(jnp.cumsum(dts(n_steps=n_steps)), layers[:, i], layers[:, i+1], color=["yellow", "blue", "green", "red", "green", "blue", "yellow"][i])
    plt.xlabel('t'), plt.ylabel('log(distance^2)')
    plt.show()

    # plot paths with closest endpoints
    lastj = -1 # last non-nan entry
    for j in range(n_steps):
        if jnp.any(jnp.isnan(xss[mini][j])):
            lastj = j-1
            break
            
    # coordinate plots
    fig, axs = plt.subplots(1, M.m)
    fig.suptitle("%s kernel, N %d, dimension %d, coordinate plots" % (kernel,N,d))
    axsflat = axs.flat if M.m > 1 else (axs,)
    for k,ax in enumerate(axsflat):
        xs = xss[mini].reshape((n_steps,M.N,M.m))
        for i in range(N):
            ax.plot(xs[:,i,k],jnp.cumsum(dts(n_steps=n_steps)))
            ax.plot(qs[mini].reshape((M.N,M.m))[i,k],0,'o',color='r',markersize=10)
            ax.plot(xs[lastj,i,k],jnp.cumsum(dts(n_steps=n_steps))[lastj],'o',color='k',markersize=10) 
            ax.set_xlabel('coord %d' % k)
            ax.set_ylabel('t')
    plt.show()


    if d == 1 and N == 2:
        points = xss[mini].reshape(-1, 1, N)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        M.newfig()
        M.plot()
        ax = plt.gca()

        norm = plt.Normalize(0,1)
        lc = LineCollection(segments, cmap='viridis', norm=norm)
        lc.set_array(jnp.cumsum(dts(n_steps=n_steps)))
        line = ax.add_collection(lc)

        plt.plot(qs[mini][0],q[0][1],'o',color='r',markersize=10)
        plt.plot(xss[mini][lastj][0],xss[mini][lastj][1],'o',color='k',markersize=10) 

        plt.gcf().colorbar(line, ax=ax)

        ax.set_xlim(xss[mini][:,0].min()-.5, xss[mini][:,0].max()+.5)
        ax.set_ylim(xss[mini][:,1].min()-.5, xss[mini][:,1].max()+.5)
        plt.title("%s kernel, N %d, dimension %d, min distance path" % (kernel,N,d))
        plt.show()

    elif d == 2:
        # plot
        M.newfig()
        M.plot()
        M.plot_path(zip(xss[mini],chartss[mini]),color=colors[mini])
        M.plotx(qs[mini],color='r')
        M.plotx((xss[mini][lastj],chartss[mini][lastj]),color='k')
        plt.title("%s kernel, N %d, dimension %d, min distance path" % (kernel,N,d))
        plt.show()
    elif d == 3:
        # plot
        M.newfig()
        M.plot()
        M.plot_path(zip(xss[mini],chartss[mini]),color=colors[mini])
        M.plotx(qs[mini],color='r')
        M.plotx((xss[mini][lastj],chartss[mini][lastj]),color='k')
        plt.title("%s kernel, N %d, dimension %d, min distance path" % (kernel,N,d))
        plt.show()


In [ ]:
# runs simulations for various kernels in various dimensions
Nsamples = 50
Ns = range(2,4+1)

# kernels
kernels = ['Gaussian','K1','K2','K3','K4']
for kernel in kernels:
    M = landmarks(2,kernel=kernel)

    from src.Riemannian import metric
    metric.initialize(M)
    from src.stochastics import Brownian_coords
    Brownian_coords.initialize(M)
    
    # number landmarks
    for N in Ns:
        # dimensions
        for d in range(1,4+1):
            print("%s kernel, N %d, dimension %d, %d sample paths" % (kernel,N,d,Nsamples))
            simBrownianDistances(d,N,Nsamples,scale=.5)